In [ ]:
%run "../includes/configuration"

In [ ]:
%run "../includes/common_functions"

In [ ]:
## now you run the variable after the command above
raw_folder_path

In [ ]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType


In [ ]:
circuits_schema = StructType(fields=[StructField("circuitId", IntegerType(), False),
                                     StructField("circuitRef", StringType(), True),
                                     StructField("name", StringType(), True),
                                     StructField("location", StringType(), True),
                                     StructField("country", StringType(), True),
                                     StructField("lat", DoubleType(), True),
                                     StructField("lng", DoubleType(), True),
                                     StructField("alt", IntegerType(), True),
                                     StructField("url", StringType(), True)

In [ ]:
circuits_df = spark.read \
.option("header", True) \
.schema(circuits_schema) \
.csv(f"{raw_folder_path}/circuits.csv")

In [ ]:
from pyspark.sql.functions import col


In [ ]:
circuits_selected_df = circuits_df.select(col("circuitId"), col("circuitRef"), col("name"), col("location"), col("country"), col("lat"), col("lng"), col("alt"))

In [ ]:
from pyspark.sql.functions import lit

In [ ]:
circuits_renamed_df = circuits_selected_df.withColumnRenamed("circuitId", "circuit_id") \
.withColumnRenamed("circuitRef", "circuit_ref") \
.withColumnRenamed("lat", "latitude") \
.withColumnRenamed("lng", "longitude") \
.withColumnRenamed("alt", "altitude") \
.withColumn("data_source", lit(v_data_source))

In [ ]:
circuits_final_df = add_ingestion_date(circuits_renamed_df)


In [ ]:
circuits_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/circuits")

In [ ]:
display(spark.read.parquet("/mnt/formula1dl/processed/circuits"))


In [ ]:
#add this message to show that it has been successfully executed
dbutils.notebook.exit("Success")